In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
!pip install kss
#한국어 문장 분리모델\

In [3]:
# 형태소 기반 토크나이징 (Konlpy)
!python3 -m pip install konlpy
!bash <(curl -s https://raw.githubusercontent.com/konlpy/konlpy/master/scripts/mecab.sh)

mecab-ko is already installed
mecab-ko-dic is already installed
mecab-python is already installed
Done.


In [4]:
!sudo apt-get install curl git
!bash <(curl -s https://raw.githubusercontent.com/konlpy/konlpy/master/scripts/mecab.sh)

Reading package lists... Done
Building dependency tree       
Reading state information... Done
curl is already the newest version (7.58.0-2ubuntu3.12).
git is already the newest version (1:2.17.1-1ubuntu0.8).
0 upgraded, 0 newly installed, 0 to remove and 57 not upgraded.
mecab-ko is already installed
mecab-ko-dic is already installed
mecab-python is already installed
Done.


In [5]:
#mecab 코랩서 오류 안나게!
!apt-get update
!apt-get install g++ openjdk-8-jdk 
!pip3 install konlpy JPype1-py3
!bash <(curl -s https://raw.githubusercontent.com/konlpy/konlpy/master/scripts/mecab.sh)

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:6 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:7 http://security.ubuntu.com/ubuntu bionic-security InRelease
Hit:9 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:11 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:12 http://archive.ubuntu.com/ubuntu bionic-updates InRelease
Hit:13 http://archive.ubuntu.com/ubuntu bionic-backports InRelease
Hit:14 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:15 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic

In [6]:
import os
import kss
import re
import pandas as pd
import numpy as np
import konlpy
from tqdm import tqdm
from konlpy.tag import Mecab

In [7]:
removal_list =  "‘, ’, ◇, ‘, ”,  ’, ', ·, \“, ·, △, ●,  , ■, (, ), \", >>, `, /, -,∼,=,ㆍ<,>, .,?, !,【,】, …, ◆,%"

def cleansing_special(sentence):
    # 특수문자를 전처리를 하는 함수
    sentence = re.sub("[.,\'\"’‘”“!?]", "", sentence)
    sentence = re.sub("[^가-힣0-9a-zA-Z\\s]", " ", sentence)
    sentence = re.sub("\s+", " ", sentence)
    
    sentence = sentence.translate(str.maketrans(removal_list, ' '*len(removal_list)))
    sentence = sentence.strip()
    
    return sentence

def cleansing_numbers(sentence):
    # 숫자를 전처리(delexicalization) 하는 함수    
    sentence = re.sub('[0-9]+', 'NUM', sentence)
    sentence = re.sub('NUM\s+', "NUM", sentence)
    sentence = re.sub('[NUM]+', "NUM", sentence)
    
    return sentence

## 데이터셋 (장르, 스토리 기반)

In [8]:
path = '/content/drive/MyDrive/data/'

df_books = pd.read_excel(os.path.join(path, 'book_preprocessing.xlsx'))

In [9]:
df_books.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 73508 entries, 0 to 73507
Data columns (total 10 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Unnamed: 0  73508 non-null  int64  
 1   title       73508 non-null  object 
 2   author      64865 non-null  object 
 3   pub         73508 non-null  object 
 4   price       73508 non-null  int64  
 5   review      73508 non-null  int64  
 6   rating      73508 non-null  float64
 7   story       67630 non-null  object 
 8   img         65530 non-null  object 
 9   genre       73508 non-null  object 
dtypes: float64(1), int64(3), object(6)
memory usage: 5.6+ MB


In [10]:
df_books.tail()

,Unnamed: 0,title,author,pub,price,review,rating,story,img,genre
73503,73503,그녀를 위해,NaN,신영미디어,2880,0,0.0,마르코 고아 조카 안젤리나 사람 필요하다 양육 위 편의 상의 아내 성실하다 착하다 ...,NaN,로맨스
73504,73504,하루만 사랑해 1,NaN,파피루스(디앤씨미디어),8100,1,7.0,날 서원 횡단보도 교통사고 당하다 상대 인천 상고 짱 노랗다 머 민 하루 놈 오히려...,NaN,로맨스
73505,73505,하루만 사랑해 2,NaN,파피루스(디앤씨미디어),8100,0,0.0,날 서원 횡단보도 교통사고 당하다 상대 인천 상고 짱 노랗다 머 민 하루 놈 오히려...,NaN,로맨스
73506,73506,서울 여자와 대구 남자가 만날 때,정민,영상노트,8100,0,0.0,NaN,NaN,로맨스
73507,73507,kiss me...,NaN,현무,7200,2,5.0,NaN,NaN,로맨스


In [11]:
df_books=df_books.rename(columns = {'Unnamed: 0' : 'book_id'})

In [12]:
#결측값 제거
df_books = df_books.dropna()

In [13]:
df_books.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 55166 entries, 0 to 65529
Data columns (total 10 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   book_id  55166 non-null  int64  
 1   title    55166 non-null  object 
 2   author   55166 non-null  object 
 3   pub      55166 non-null  object 
 4   price    55166 non-null  int64  
 5   review   55166 non-null  int64  
 6   rating   55166 non-null  float64
 7   story    55166 non-null  object 
 8   img      55166 non-null  object 
 9   genre    55166 non-null  object 
dtypes: float64(1), int64(3), object(6)
memory usage: 4.6+ MB


In [14]:
df_books.value_counts(df_books['rating'].values, sort=False)

0.0     44010
2.0        57
3.0        13
3.5         1
3.7         1
        ...  
9.6       174
9.7        78
9.8        87
9.9        15
10.0     1273
Length: 61, dtype: int64

In [15]:
df_books.value_counts(df_books['review'].values, sort=False)

0      44010
1       3231
2       1347
3        902
4        661
       ...  
244        1
264        1
268        1
311        1
326        1
Length: 157, dtype: int64

In [16]:
#레이팅, 리뷰값 모두 0점인것 제거 (다행히 리뷰와 레이팅이 모두 0점인게 공통적이다)
df_books =df_books[df_books.review != 0]

In [17]:
df_books=df_books.reset_index(drop = False, inplace = False)

In [18]:
df_books.tail()

,index,book_id,title,author,pub,price,review,rating,story,img,genre
11151,65517,65517,범상치 않은 관계 1,정해길,다향,8100,1,8.0,여자 키 중성 외모 만년 휴학생 송혜민 아버지 빚 때문 만난 사채 업자 로부터 가지...,http://image.yes24.com/goods/11367513/L,로맨스
11152,65524,65524,보통의 세계,권도란,스칼렛,8820,1,6.0,사고 동시 살의 기억 남자 승 왜 그 일 년 기억 왜 열렬하다 사랑 애인 유라 것 ...,http://image.yes24.com/goods/11318506/L,로맨스
11153,65525,65525,미주부동산 2,이휴정,신영미디어,8100,1,4.0,그 아프다 몸 끌 때 생각 언제 이 사람 내 눈물 때 다른 것 제 이렇다 식 저 자...,http://image.yes24.com/goods/11245157/L,로맨스
11154,65526,65526,미주부동산 1,이휴정,신영미디어,8100,1,4.0,김 콜라 얼음 아이스커피 소리 작다 노래방 마이크 저녁 간판 자동차 달리 도시 셰익...,http://image.yes24.com/goods/11245152/L,로맨스
11155,65528,65528,나쁜 결혼,김진영,스칼렛,8100,1,4.0,저 결혼 아내 로서 의무 사랑 마음 여자 지우 그렇다 남자 아내 당신 말 의무 속 ...,http://image.yes24.com/sysimage/pd_19_L.gif,로맨스


## 장르별 분리

In [19]:
df_mystery=df_books[df_books['genre']=='추리/미스터리']
df_horror =df_books[df_books['genre']== '공포/스릴러']
df_fantasy = df_books[df_books['genre']=='판타지']
df_martial = df_books[df_books['genre']=='무협']
df_sf = df_books[df_books['genre']=='SF']
df_history = df_books[df_books['genre']=='역사']
df_romance = df_books[df_books['genre']=='로맨스']

In [20]:
print(df_mystery.shape)
print(df_horror.shape)
print(df_fantasy.shape)
print(df_martial.shape)
print(df_sf.shape)
print(df_history.shape)
print(df_romance.shape)

(2489, 11)
(1230, 11)
(2399, 11)
(470, 11)
(656, 11)
(1324, 11)
(2588, 11)


## 형태소 분리작업

In [21]:
from konlpy.tag import Mecab

In [22]:

df_mystery_list = list(np.array(df_mystery['story'].tolist()))
df_horror_list = list(np.array(df_horror['story'].tolist()))
df_fantasy_list = list(np.array(df_fantasy['story'].tolist()))
df_martial_list = list(np.array(df_martial['story'].tolist()))
df_sf_list = list(np.array(df_sf['story'].tolist()))
df_history_list = list(np.array(df_history['story'].tolist()))
df_romance_list = list(np.array(df_romance['story'].tolist()))

In [23]:
mecab = Mecab()

In [24]:
#미스터리
all_sentences_mystery = []
for idx, line in enumerate(df_mystery_list):
  #print(f"--미스테리소설{idx}번--")
  preprocessed = cleansing_numbers(line)
  preprocessed = cleansing_special(preprocessed)
  #명사
  preprocessed_list = ' '.join(list(set(mecab.nouns(preprocessed))))
  #print(preprocessed_list)
  all_sentences_mystery.append(preprocessed_list)

In [25]:
#호러
all_sentences_horror = []
for idx, line in enumerate(df_horror_list):
  #print(f"--호러소설{idx}번--")
  preprocessed = cleansing_numbers(line)
  preprocessed = cleansing_special(preprocessed)
  #명사
  preprocessed_list = ' '.join(list(set(mecab.nouns(preprocessed))))
  #print(preprocessed_list)
  all_sentences_horror.append(preprocessed_list)

In [26]:
#판타지
all_sentences_fantasy = []
for idx, line in enumerate(df_fantasy_list):
  #print(f"--판타지소설{idx}번--")
  preprocessed = cleansing_numbers(line)
  preprocessed = cleansing_special(preprocessed)
  #명사
  preprocessed_list = ' '.join(list(set(mecab.nouns(preprocessed))))
  #print(preprocessed_list)
  all_sentences_fantasy.append(preprocessed_list)

In [27]:
#무협
all_sentences_martial = []
for idx, line in enumerate(df_martial_list):
  #print(f"--무협소설{idx}번--")
  preprocessed = cleansing_numbers(line)
  preprocessed = cleansing_special(preprocessed)
  #명사
  preprocessed_list = ' '.join(list(set(mecab.nouns(preprocessed))))
  #print(preprocessed_list)
  all_sentences_martial.append(preprocessed_list)

In [28]:
#sf
all_sentences_sf = []
for idx, line in enumerate(df_sf_list):
  #print(f"--sf소설{idx}번--")
  preprocessed = cleansing_numbers(line)
  preprocessed = cleansing_special(preprocessed)
  #명사
  preprocessed_list = ' '.join(list(set(mecab.nouns(preprocessed))))
  #print(preprocessed_list)
  all_sentences_sf.append(preprocessed_list)

In [29]:
#역사
all_sentences_history = []
for idx, line in enumerate(df_history_list):
  #print(f"--역사소설{idx}번--")
  preprocessed = cleansing_numbers(line)
  preprocessed = cleansing_special(preprocessed)
  #명사
  preprocessed_list = ' '.join(list(set(mecab.nouns(preprocessed))))
  #print(preprocessed_list)
  all_sentences_history.append(preprocessed_list)

In [30]:
#로맨스
all_sentences_romance = []
for idx, line in enumerate(df_romance_list):
  #print(f"--연애소설{idx}번--")
  preprocessed = cleansing_numbers(line)
  preprocessed = cleansing_special(preprocessed)
  #명사
  preprocessed_list = ' '.join(list(set(mecab.nouns(preprocessed))))
  #print(preprocessed_list)
  all_sentences_romance.append(preprocessed_list)

## TF-IDF 알고리즘

In [31]:
from sklearn.feature_extraction.text import TfidfVectorizer
from collections import defaultdict  #단어 개수를 세고, 빈도와 리스트 셋 항목 정리리

In [32]:
print(len(all_sentences_mystery))
print(len(all_sentences_horror))
print(len(all_sentences_fantasy))
print(len(all_sentences_martial))
print(len(all_sentences_sf))
print(len(all_sentences_history))
print(len(all_sentences_romance))

2489
1230
2399
470
656
1324
2588


In [33]:
#미스터리
tfidf_vectorizer = TfidfVectorizer()
tfidf_matrix = tfidf_vectorizer.fit_transform(all_sentences_mystery)

word2id = defaultdict(lambda : 0)
for idx, feature in enumerate(tfidf_vectorizer.get_feature_names()):
  word2id[feature] = idx

In [34]:
#tfidf_vectorizer.vocabulary_  #어떤 단어가 어떤 번호를 할당받았는지!

In [35]:
tfidf_matrix.shape

(2489, 9006)

In [38]:
for idx, sent in enumerate(all_sentences_mystery):
  #print(f'---미스테리 소설 {idx} 번---')
  results_mystery = [(token, tfidf_matrix[idx, word2id[token]]) for token in sent.split()]
  results_mystery.sort(key = lambda element : element[1], reverse = True)
  #print(results_mystery)
  #print("\n")

In [39]:
#호러
tfidf_vectorizer = TfidfVectorizer()
tfidf_matrix = tfidf_vectorizer.fit_transform(all_sentences_horror)

word2id = defaultdict(lambda : 0)
for idx, feature in enumerate(tfidf_vectorizer.get_feature_names()):
  word2id[feature] = idx

In [40]:
for idx, sent in enumerate(all_sentences_horror):
  #print(f'---공포 소설 {idx} 번---')
  results_horror = [(token, tfidf_matrix[idx, word2id[token]]) for token in sent.split()]
  results_horror.sort(key = lambda element : element[1], reverse = True)
  #print(results_horror)
  #print("\n")

In [41]:
#판타지
tfidf_vectorizer = TfidfVectorizer()
tfidf_matrix = tfidf_vectorizer.fit_transform(all_sentences_fantasy)

word2id = defaultdict(lambda : 0)
for idx, feature in enumerate(tfidf_vectorizer.get_feature_names()):
  word2id[feature] = idx

In [42]:
for idx, sent in enumerate(all_sentences_fantasy):
  #print(f'--- 판타지 소설 {idx} 번---')
  results_fantasy = [(token, tfidf_matrix[idx, word2id[token]]) for token in sent.split()]
  results_fantasy.sort(key = lambda element : element[1], reverse = True)
  #print(results_fantasy)
  #print("\n")

In [50]:
#무협
tfidf_vectorizer = TfidfVectorizer()
tfidf_matrix = tfidf_vectorizer.fit_transform(all_sentences_martial)

word2id = defaultdict(lambda : 0)
for idx, feature in enumerate(tfidf_vectorizer.get_feature_names()):
  word2id[feature] = idx

In [51]:
for idx, sent in enumerate(all_sentences_martial):
  #print(f'--- 무협 소설 {idx} 번---')
  results_martial = [(token, tfidf_matrix[idx, word2id[token]]) for token in sent.split()]
  results_martial.sort(key = lambda element : element[1], reverse = True)
  #print(results_martial)
  #print("\n")

In [53]:
#sf
tfidf_vectorizer = TfidfVectorizer()
tfidf_matrix = tfidf_vectorizer.fit_transform(all_sentences_sf)

word2id = defaultdict(lambda : 0)
for idx, feature in enumerate(tfidf_vectorizer.get_feature_names()):
  word2id[feature] = idx

In [54]:
for idx, sent in enumerate(all_sentences_sf):
  #print(f'--- sf 소설 {idx} 번---')
  results_sf = [(token, tfidf_matrix[idx, word2id[token]]) for token in sent.split()]
  results_sf.sort(key = lambda element : element[1], reverse = True)
  #print(results_sf)
  #print("\n")

In [55]:
#역사
tfidf_vectorizer = TfidfVectorizer()
tfidf_matrix = tfidf_vectorizer.fit_transform(all_sentences_history)

word2id = defaultdict(lambda : 0)
for idx, feature in enumerate(tfidf_vectorizer.get_feature_names()):
  word2id[feature] = idx

In [56]:
for idx, sent in enumerate(all_sentences_history):
  #print(f'--- sf 소설 {idx} 번---')
  results_history = [(token, tfidf_matrix[idx, word2id[token]]) for token in sent.split()]
  results_history.sort(key = lambda element : element[1], reverse = True)
  #print(results_history)
  #print("\n")

In [57]:
#연애
tfidf_vectorizer = TfidfVectorizer()
tfidf_matrix = tfidf_vectorizer.fit_transform(all_sentences_romance)

word2id = defaultdict(lambda : 0)
for idx, feature in enumerate(tfidf_vectorizer.get_feature_names()):
  word2id[feature] = idx

In [60]:
for idx, sent in enumerate(all_sentences_romance):
  #print(f'--- 연애 소설 {idx} 번---')
  results_romance = [(token, tfidf_matrix[idx, word2id[token]]) for token in sent.split()]
  results_romance.sort(key = lambda element : element[1], reverse = True)
  #print(results_romance)
  #print("\n")